In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
!pip install pandas
!pip install JPype1==0.7.0
!pip install gensim==3.8.3
!pip install pyLDAvis==3.2.2
!pip install networkx
!pip install google-cloud-language
!apt-get update -qq
!apt-get install fonts-nanum* -qq


import json
import pandas as pd
import nltk
nltk.download('all')
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import ast
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim as gensimvis
from collections import defaultdict
import networkx as nx
import operator
from matplotlib import font_manager as fm
from matplotlib import rc
from tqdm import tqdm
from google.cloud import language_v1
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=""


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 2s (5,586 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 123105 files and di

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [ ]:
import os
import nltk
import pandas as pd
from google.cloud import language_v1
import time

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""

# 감성분석 함수 정의
def analyze_sentiment(text_content):
    client = language_v1.LanguageServiceClient()
    document = language_v1.Document(content=text_content, type_=language_v1.Document.Type.PLAIN_TEXT)
    response = client.analyze_sentiment(request={'document': document})
    sentiment = response.document_sentiment.score
    return sentiment

# 크롤링 데이터 파일 경로 설정
file_path = '크롤링완료_100개씩.csv'

# CSV 파일 읽기
data = pd.read_csv(file_path)

# 'Content' 열에서 리뷰 추출
reviews = data['Content']

# 감성분석 결과 저장할 열 추가
data['Sentiment'] = None

# 리뷰 감성분석 수행
for i, review in enumerate(reviews[:10000]):
    if isinstance(review, str):  # 문자열인 경우에만 감성분석 수행
        # Rate limiting - pause for 0.1 seconds (10 requests per second)
        time.sleep(0.1)
        print(f"Analyzing sentiment for review {i + 1}...")
        sentiment = analyze_sentiment(review)
        data.at[i, 'Sentiment'] = sentiment
    else:  # 결측치인 경우에는 0으로 처리 (또는 다른 방식으로 처리 가능)
        data.at[i, 'Sentiment'] = 0

# 결과 CSV 파일로 저장
output_path = '감성분석_결과_1.csv'
data.to_csv(output_path, index=False)

print(f"감성분석 결과가 '{output_path}'에 저장되었습니다.")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Analyzing sentiment for review 4826...
Analyzing sentiment for review 4827...
Analyzing sentiment for review 4828...
Analyzing sentiment for review 4829...
Analyzing sentiment for review 4830...
Analyzing sentiment for review 4831...
Analyzing sentiment for review 4832...
Analyzing sentiment for review 4833...
Analyzing sentiment for review 4834...
Analyzing sentiment for review 4835...
Analyzing sentiment for review 4836...
Analyzing sentiment for review 4837...
Analyzing sentiment for review 4838...
Analyzing sentiment for review 4839...
Analyzing sentiment for review 4840...
Analyzing sentiment for review 4841...
Analyzing sentiment for review 4842...
Analyzing sentiment for review 4843...
Analyzing sentiment for review 4844...
Analyzing sentiment for review 4845...
Analyzing sentiment for review 4846...
Analyzing sentiment for review 4847...
Analyzing sentiment for review 4848...
Analyzing sentiment for review 4849...
Analyzing sentiment for revi

In [ ]:
import os
import nltk
import pandas as pd
from google.cloud import language_v1
import time

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""

# 감성분석 함수 정의
def analyze_sentiment(text_content):
    try:
        client = language_v1.LanguageServiceClient()
        document = language_v1.Document(content=text_content, type_=language_v1.Document.Type.PLAIN_TEXT)
        response = client.analyze_sentiment(request={'document': document})
        sentiment = response.document_sentiment.score
        return sentiment
    except Exception as e:
        print(f"Error analyzing sentiment: {str(e)}")
        return None

# 크롤링 데이터 파일 경로 설정
file_path = '크롤링완료_100개씩.csv'

# CSV 파일 읽기
data = pd.read_csv(file_path)

# 'Content' 열에서 리뷰 추출
reviews = data['Content']

# 감성분석 결과 저장할 열 추가
data['Sentiment'] = None

# 리뷰 감성분석 수행
for i, review in enumerate(reviews[24564:], start=24564):
    if isinstance(review, str):  # 문자열인 경우에만 감성분석 수행
        # Rate limiting - pause for 0.1 seconds (10 requests per second)
        time.sleep(0.1)
        print(f"Analyzing sentiment for review {i + 1}...")
        sentiment = analyze_sentiment(review)
        if sentiment is not None:
            data.at[i, 'Sentiment'] = sentiment
    else:  # 결측치인 경우에는 0으로 처리 (또는 다른 방식으로 처리 가능)
        data.at[i, 'Sentiment'] = 0

# 결과 CSV 파일로 저장
output_path = '감성분석_결과_3.csv'
data.to_csv(output_path, index=False)

print(f"감성분석 결과가 '{output_path}'에 저장되었습니다.")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Analyzing sentiment for review 25280...
Analyzing sentiment for review 25281...
Analyzing sentiment for review 25282...
Analyzing sentiment for review 25283...
Analyzing sentiment for review 25284...
Analyzing sentiment for review 25285...
Analyzing sentiment for review 25286...
Analyzing sentiment for review 25287...
Analyzing sentiment for review 25288...
Analyzing sentiment for review 25289...
Analyzing sentiment for review 25290...
Analyzing sentiment for review 25291...
Analyzing sentiment for review 25292...
Analyzing sentiment for review 25293...
Analyzing sentiment for review 25294...
Analyzing sentiment for review 25295...
Analyzing sentiment for review 25296...
Analyzing sentiment for review 25297...
Analyzing sentiment for review 25298...
Analyzing sentiment for review 25299...
Analyzing sentiment for review 25300...
Analyzing sentiment for review 25301...
Analyzing sentiment for review 25302...
Analyzing sentiment for review 25303...
Anal

In [ ]:
  # 결과 CSV 파일로 저장
output_path = '감성분석_결과_3.csv'
data.to_csv(output_path, index=False)

print(f"감성분석 결과가 '{output_path}'에 저장되었습니다.")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


감성분석 결과가 '감성분석_결과_3.csv'에 저장되었습니다.


In [ ]:
df= pd.read_csv('감성분석_결과.csv')
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Name,URL,Tourist_destination,Author_Name,ID,Content,Keywords,Image Links,Sentiment
0,1.0,부산에뜬고등어,https://pcmap.place.naver.com/restaurant/14790...,삼락생태공원,봄날36,NaN,처음 방문해 본 곳인데 부산 사상구 괘법동 맛집은 화덕 부산에 뜬 고등어는 정말로 ...,['음식이 맛있어요'],https://search.pstatic.net/common/?autoRotate=...,0.3
1,2.0,부산에뜬고등어,https://pcmap.place.naver.com/restaurant/14790...,삼락생태공원,망고토끼님,NaN,생선구이 한상이 너무 정갈하게 나와서 부모님 모시고 가기 딱 좋을것 같아요! 이른아...,"['음식이 맛있어요', '양이 많아요', '특별한 메뉴가 있어요', '가성비가 좋아...",https://search.pstatic.net/common/?autoRotate=...,0.8
2,3.0,부산에뜬고등어,https://pcmap.place.naver.com/restaurant/14790...,삼락생태공원,잘먹는여행자,NaN,화덕 생선구이 정말 맛있게 잘먹었어용. 모든 음식이 짜지않고 간간하이 딱 좋습니다!...,"['매장이 넓어요', '매장이 청결해요', '주차하기 편해요', '친절해요', '음...",https://search.pstatic.net/common/?autoRotate=...,0.9
3,4.0,부산에뜬고등어,https://pcmap.place.naver.com/restaurant/14790...,삼락생태공원,자라밥,NaN,요근래 고기를 자주 먹다보니 생선구이가 땡기는 날이라 찾아보니 사상역 근처에 새로 ...,"['음식이 맛있어요', '매장이 넓어요', '친절해요', '화장실이 깨끗해요', '...",https://search.pstatic.net/common/?autoRotate=...,0.7
4,5.0,부산에뜬고등어,https://pcmap.place.naver.com/restaurant/14790...,삼락생태공원,뎀잇31,NaN,사상에 사는 사람들 부럽다 이런 밥집 많아서...약간 세미 한식 뷔페 같은 느낌 들...,"['양이 많아요', '음식이 맛있어요', '가성비가 좋아요', '재료가 신선해요',...",https://search.pstatic.net/common/?autoRotate=...,0.9
...,...,...,...,...,...,...,...,...,...,...
81030,NaN,서울왕돈까스,https://pcmap.place.naver.com/restaurant/14544...,부산아시아드조각광장,행복하자3753,NaN,생선가츠 양 엄청 많아요 맛있게 잘 먹었습니다^^,NaN,NaN,NaN
81031,NaN,서울왕돈까스,https://pcmap.place.naver.com/restaurant/14544...,부산아시아드조각광장,ami****,NaN,맛나요^^,NaN,NaN,NaN
81032,NaN,서울왕돈까스,https://pcmap.place.naver.com/restaurant/14544...,부산아시아드조각광장,미소선이,NaN,맛있어요 친절해요 해장으로 딱이에요 겉만 보고 판단 하지 마세요,NaN,NaN,NaN
81033,NaN,서울왕돈까스,https://pcmap.place.naver.com/restaurant/14544...,부산아시아드조각광장,빛바랜교복,NaN,맛있어요,NaN,NaN,NaN


In [ ]:
import pandas as pd

# 가게소개용.csv 파일을 데이터프레임으로 읽어옵니다. 인코딩을 'ansi'로 설정합니다.
df = pd.read_csv('가게소개용.csv', encoding='cp949')

# 'Tourist destination' 컬럼의 유니크값을 추출합니다.
unique_destinations = df['Tourist destination'].unique()

# 여행지소개용.csv 파일을 데이터프레임으로 읽어옵니다.
df_destinations = pd.read_csv('여행지소개용.csv')

# 'Tourist destination' 컬럼에 유니크값을 저장합니다.
df_destinations['Tourist destination'] = unique_destinations

# 결과를 저장합니다.
df_destinations.to_csv('여행지소개용.csv', index=False)


In [ ]:
# 가게소개용.csv 파일을 데이터프레임으로 읽어옵니다. 인코딩을 'ansi'로 설정합니다.
df = pd.read_csv('가게소개용.csv', encoding='cp949')

# 'Tourist destination' 컬럼의 유니크값을 추출합니다.
unique_destinations = df['Tourist destination'].unique()
unique_destinations

array(['삼락생태공원', '부산서부사상버스터미널', '부산시민공원', '서면시장', '전포카페거리', '화명생태공원',
       '해운대블루라인파크 미포정거장', '해리단길', '신세계백화점 센텀시티점', '광안리해수욕장', '민락수변공원',
       '유엔평화기념관', '범어사', '금강식물원', '회동마루', '금정산성', '황령산', '연산동고분군',
       '감천문화마을', '을숙도', '부산송도해수욕장', '암남공원', '비프광장', '부산타워', '차이나타운',
       '유치환의 우체통', '이중섭거리', '문화공감수정', '흰여울문화마을', '부산대교', '온천천카페거리', '충렬사',
       '부산아시아드조각광장'], dtype=object)

In [ ]:
# import flask_cors from CORS
# CORS(app)
# from flask import Flask, request, jsonify, make_response
# import json
# import concurrent.futures
# from functools import lru_cache
# import flask_cors from CORS
# import openai
# import pandas as pd
#
#
# CORS(app)
#
# app = Flask(__name__)

from flask import Flask, request, jsonify, make_response
import json
import concurrent.futures
from functools import lru_cache
import openai
import pandas as pd

app = Flask(__name__)

# OpenAI GPT API 키
openai.api_key = ""

# 데이터 로드
travel_data = pd.read_csv('여행지소개용1.csv')
user_data = pd.read_csv('User.csv')
store_data = pd.read_csv('가게소개용.csv')

# 사용자 정보 가져오기
def get_user_info(user_id):
    try:
        user_id = int(user_id)
    except ValueError:
        return None, None

    user = user_data[user_data['ID'] == user_id]
    if not user.empty:
        return user.iloc[0]['Age'], user.iloc[0]['Gender']
    else:
        return None, None



# 여행지 적합도 평가 함수
@lru_cache(maxsize=None)  # 캐싱 기능 추가
def get_destination_score_and_reason(age, gender, destination_name, destination_description, image_link):
    # API 호출을 사용하여 여행지에 대한 적합도를 평가하고, 그 이유를 반환합니다.
    prompt = f"""
    고객의 나이: {age},
    고객의 성별: {gender},
    여행지 이름: {destination_name},
    여행지 설명: {destination_description}

    이 정보를 기반으로 여행지의 적합도를 1부터 10까지의 정수로 평가하십시오.
    """

    response = openai.Completion.create(engine="text-davinchi-003", prompt=prompt, max_tokens=10)
    try:
        score = int(response.choices[0].text.strip())
    except ValueError:
        score = 0

    prompt_reason = [
        {
            "role": "system",
            "content": "you are a travel expert"
        },
        {
            "role": "user",
            "content": f"고객의 나이: {age}, 고객의 성별: {gender}, 관광명소: {destination_name}, 관광명소에 대한 설명: {destination_description}. 고객에게 이 관광명소를 추천하는 말을 작성해줘."
        }
    ]

    reason_response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=prompt_reason, max_tokens=120)
    reason = reason_response.choices[0].message['content']

    return score, reason, image_link, destination_name  # destination_name을 반환 값에 추가합니다.


# 여행지 추천 함수
def recommend_destination(age, gender, user_request):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Prepare arguments for get_destination_score_and_reason
        args = [(age, gender, row['Tourist destination'], row['description'], row['image_link']) for _, row in travel_data.iterrows()]
        # Use map function to execute the function in parallel
        destination_scores = list(executor.map(lambda x: get_destination_score_and_reason(*x), args))

    # 점수 내림차순으로 정렬
    destination_scores.sort(key=lambda x: x[0], reverse=True)

    # 상위 3개 여행지 선택
    top_3_destinations = destination_scores[:3]

    # 추천 정보 생성
    recommendations = []
    for i, (score, reason, image_link, destination_name) in enumerate(top_3_destinations):  # destination_name을 unpack
        recommendation = {
            'rank': i + 1,
            'Tourist destination': destination_name,  # 여기서 destination_name 변수 사용
            'score': score,
            'reason': reason,
            'image_link': image_link
        }
        recommendations.append(recommendation)

    return recommendations


def build_preflight_response():
    response = make_response()
    response.headers.add("Access-Control-Allow-Origin", "*")
    response.headers.add('Access-Control-Allow-Headers', "*")
    response.headers.add('Access-Control-Allow-Methods', "*")
    return response


def build_actual_response(response):
    response.headers.add("Access-Control-Allow-Origin", "*")
    return response


# Flask route for the home page
@app.route('/', methods=['OPTIONS','POST'])
def recommendations():
    user_id = request.args.get('user_id')
    user_request = request.args.get('user_request')
    if request.method == 'OPTIONS':
        return build_preflight_response()
    print(user_id)
    print(user_request)
    age, gender = get_user_info(user_id)

    if age is not None and gender is not None:
        recommendations = recommend_destination(age, gender, user_request)
        print(jsonify(recommendations))
        return jsonify(recommendations)
    else:
        return jsonify({"error": "사용자를 찾을 수 없습니다."}), 404


@app.route('/stores', methods=['OPTIONS','GET'])
def stores():
    if request.method == 'OPTIONS':
        return build_preflight_response()
    destination = request.args.get('destination')
    print(destination)
    # 선택된 여행지를 기반으로 가게 데이터 필터링
    filtered_data = store_data[store_data['Tourist_destination'] == destination]

    # 필터링된 데이터를 JSON 형식으로 변환
    return jsonify(filtered_data.to_dict(orient='records'))


if __name__ == "__main__":
    app.run(debug=True)